In [2]:
import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os
import pandas as pd
import time

from datetime import datetime
import shutil

/Users/callinswitzer/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
os.chdir("/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning")

In [8]:
# get paths for videos

vidPaths = []

for path, subdirs, files in os.walk(os.getcwd()):
    for name in files:
        if name.endswith(".mp4"):
            vidPaths.append(os.path.join(path, name))
vidPaths

['/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue2_28Nov2016_Hive4_high_compressed/BeeBlue2_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue3_28Nov2016_Hive4_high_compressed/BeeBlue3_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue4_28Nov2016_Hive4_high_compressed/BeeBlue4_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue5_28Nov2016_Hive4_high_compressed/BeeBlue5_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue6_28Nov2016_Hive4_high_compressed/BeeBlue6_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/BeeBlue7_28Nov2016_Hive4_high_compressed/BeeBlue7_28Nov2016_Hive4_high_compressed.mp4',
 '/Users/callinswitzer/Desktop/CompressedBeeFrequencyLearning/Beeblue8_29Nov2016_Hive4_high_compress

In [46]:
# load a video
ii = 3
aa = vidPaths[ii]
cap = cv2.VideoCapture(aa)
numFr = int(round(cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)))
print 'numFrames = ' + str(numFr)

numFrames = 117


In [47]:
cap = cv2.VideoCapture(aa)
ctr = 0

# reset frame num to zero
cap.set(cv2.cv.CV_CAP_PROP_FRAME_COUNT, 0) 

frames = []

while(ctr < numFr):  # check !
    # capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    frames.append(gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    print ctr
    ctr += 1
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [405]:
plt.clf()
io.imshow(frames[0])

In [50]:
frames_arr = np.array(frames)

In [51]:
frames_arr.shape

(117, 1024, 1280)

In [52]:
bkg = np.median(frames_arr,axis = 0)

In [53]:
bkg.shape

(1024, 1280)

In [54]:
plt.clf()
io.imshow(bkg, cmap = "Greys")

In [93]:
# make bkg-subtracted images
bkg_sub = np.abs(frames_arr - bkg )

In [94]:
bkg_sub.shape

(117, 1024, 1280)

In [244]:
imnum = 79

In [280]:
imnum +=1

plt.clf()
io.imshow(frames[imnum])

In [281]:
# this might be the way to go

normimg = bkg_sub[imnum] / 255.0

resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)

In [269]:

plt.clf()
io.imshow(bkg_sub[imnum], cmap = "Greys")

In [270]:
# convert to BW
im_bw = (bkg_sub[imnum] > 10) * 1.0

In [271]:
im_bw.shape

(1024, 1280)

In [272]:
plt.clf()
io.imshow(im_bw)

In [273]:
# clean up image
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
erosion = cv2.erode(im_bw,kernel,iterations = 1)
plt.clf()
io.imshow(erosion)

In [274]:
dilation = cv2.dilate(erosion, kernel, iterations = 1)


plt.clf()
io.imshow(dilation)

In [275]:
# # crop image
# def submatrix(arr):
#     x, y = np.nonzero(arr)
#     # Using the smallest and largest x and y indices of nonzero elements, 
#     # we can find the desired rectangular bounds.  
#     # And don't forget to add 1 to the top bound to avoid the fencepost problem.
#     return arr[x.min():x.max()+1, y.min():y.max()+1]

# cpdImg = submatrix(dilation)



In [276]:
# reduce resolution

resized_image = cv2.resize(dilation, (128, 102)) 
plt.clf()
io.imshow(resized_image)

In [277]:
# this might be the way to go

normimg = bkg_sub[imnum] / 255.0

resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)


In [381]:
# do clustering

os.chdir("/Users/callinswitzer/Desktop/imgExamples")

smallFrames = []

for ii in range(117):
    normimg = bkg_sub[ii] 
    norming2 = normimg[200:, :] # remove portion with text

    resized_image = cv2.resize(normimg, (128, 82)) 
    smallFrames.append(resized_image)
    


In [382]:
plt.clf()
norming2 = normimg[200:, :]
io.imshow(norming2)
norming2.shape

(824, 1280)

In [383]:
plt.clf()
io.imshow(smallFrames[0])

In [384]:
smallFrames = np.array(smallFrames)
smallFrames.shape

(117, 82, 128)

In [386]:
# reshape array for machine learning -- each row is a photo
fr_resh = np.reshape(smallFrames, (117, 82*128))
fr_resh.shape


(117, 10496)

In [387]:
# knn
from sklearn.cluster import KMeans


In [388]:
X = fr_resh
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       2, 1], dtype=int32)

In [389]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [390]:
X.std(axis=0)

np.histogram(X[1,:])

(array([10168,   207,    63,    25,    19,     6,     2,     2,     2,     2]),
 array([   0.        ,   22.60853653,   45.21707306,   67.82560959,
          90.43414612,  113.04268265,  135.65121918,  158.25975571,
         180.86829224,  203.47682877,  226.0853653 ]))

In [391]:
# normalize X matrix
# X = preprocessing.scale(X)

In [392]:
pca = PCA(n_components=2)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [393]:
X2 = pca.transform(X)
y = kmeans.labels_


In [394]:
X2.shape

(117, 2)

In [397]:
plt.clf()
plt.scatter(X2[:, 0], X2[:,1], c=y, s= 100)

n = range(117)
for i, txt in enumerate(n):
    plt.annotate(txt, (X2[:, 0][i] + 5, X2[:,1][i] + 5))

In [349]:
plt.clf()
plt.scatter(X2[:, 0], X2[:,1], c=y)
plt.show()

In [403]:

io.imshow(frames[116])

In [404]:
plt.close("all")

In [287]:
resized_image = cv2.resize(normimg, (128, 102)) 
plt.clf()
io.imshow(resized_image)


In [289]:
cv2.imwrite(str(ii) + ".png", normimg)
    

True